# Step 3  gaze processing


This code was developed and written originally by Debora Nolte. Then extended and adapted by Jasmin L. Walter

In [206]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Preprocessing

## import

In [1]:
import copy  # copy big/deep objects by value
import csv
import datetime  # datetime operations
import itertools  # operate with iterators
import json  # read/write from/into json format
import math
import os  # OS operations (read/write files/folders)
import sys
import time
import warnings  # hide warnings
from collections import Counter
from itertools import groupby
import matplotlib

# process parallelization
from multiprocessing import Manager, Pool, RawArray, cpu_count
from os.path import exists

import matplotlib.pyplot as plt  # mother of plots focr Python

# import mlxtend
import numpy as np  # array/matrix operations (e.g. linear algebra)
import pandas as pd  # operate with dataframes
# import pyxdf  # read XDF files (LSL streams recordings)

import scipy.stats
import seaborn as sns  # matplotlib plotting nice with shortcuts
from IPython.display import Markdown, display  # print nicely
from ipywidgets import IntProgress
#from matplotlib.pyplot import cm
from scipy.signal import savgol_coeffs
from tqdm.notebook import tqdm, trange  # mother of progressbars
from scipy.stats import ks_2samp
from multiprocessing import Pool, cpu_count

import scipy.signal

## optimization

In [2]:
# warnings.simplefilter(action="ignore", category=FutureWarning)

# raw and processed data paths
# PATH_RAW = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/27.07.23/data"
# PATH_PROC = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/EEG_Data_Skripte_Debbie/Events"
# PATH_FOREYE = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/EEG_Data_Skripte_Debbie/ET_Output_MAD-sacc"
# PATH_TRG = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/EEG_Data_Skripte_Debbie/TriggerFiles_fEEG"


# specify decimals format on pandas tables
# pd.options.display.float_format = "{:.5f}".format

# inline static plotting (default)
%matplotlib inline
# interactive plotting
# %matplotlib widget

# progress bar customized format
B_FORMAT = """📄 {n_fmt} of {total_fmt} {desc} processed: {bar} 
            {percentage:3.0f}% ⏱️{elapsed} ⏳{remaining} ⚙️{rate_fmt}{postfix}"""


CORES = cpu_count()  # number of cpu threads for multiprocessing
print(f"Total CPU threads: {CORES}")


def pbar_fork_hack():
    """
    Hack to enforce progress bars to be displayed by fork processes on
    IPython Apps like Jupyter Notebooks.

    Avoids [IPKernelApp] WARNING | WARNING: attempted to send message from fork

    Important: pass this function as argument for the initializer parameter
    while initializing a multiprocessing pool to make it work. E.g.:

    pool = Pool(processes=N_CORES, initializer=pbar_fork_hack)

    Source:
     - https://github.com/ipython/ipython/issues/11049#issue-306086846
     - https://github.com/tqdm/tqdm/issues/485#issuecomment-473338308
    """
    print(" ", end="", flush=True)

Total CPU threads: 8


## data paths & participant list

In [3]:
data_savepath_resampled = "D:/Jasmin/Human_A_Data/Experiment1/Exploration_short/pre-processing/velocity_based/step2_resampling"


data_savepath_intermediateP = "D:/Jasmin/Human_A_Data/Experiment1/Exploration_short/pre-processing/velocity_based/step3_gazeProcessing/intermediateProcessing"
data_savepath_intervals = "D:/Jasmin/Human_A_Data/Experiment1/Exploration_short/pre-processing/velocity_based/step3_gazeProcessing/intervals"
data_savepath_processed_gazes = "D:/Jasmin/Human_A_Data/Experiment1/Exploration_short/pre-processing/velocity_based/step3_gazeProcessing"

# Getting the Folder without hidden files in ascending order 
# DATA_FOLDER = sorted([f for f in os.listdir(DATA_PATH) if not f.startswith('.')], key=str.lower)
# PROCESSED_DATA_FOLDER = sorted([f for f in os.listdir(PROCESSED_DATA_PATH) if not f.startswith('.')], key=str.lower)

# savepath = r'F:\WestbrookProject\SpaRe_Data\pre-processing_2023\step1_preparation\'
# os.chdir(r'F:\WestbrookProject\SpaRe_Data\pre-processed_csv\')

In [4]:
# identify all participants from folder, use this code instead of the participant list based approach
# subIDs = []
# for sub in DATA_FOLDER:
#     if sub[0:4].isdigit() and sub.startswith('1'):
#         subIDs.append(int(sub[0:4]))
#     else:
#         pass
# subIDs = np.unique(subIDs)
# print(subIDs)

In [5]:
# Participant list of all participants that participated 5 sessions x 30 min
# in Westbrook city - HumanA version with agents
PartList = [365, 1754, 2258, 2693, 3310, 4176, 4597, 4796, 4917, 5741, 6642, 7093, 7264, 7412, 7842, 8007, 8469, 8673, 9472, 9502, 9586, 9601]

print(len(PartList),'participants') 

26 participants


In [6]:
# # custom participant list, if only a subset of participants should be processed
# PartList = [1008]

## test code

In [7]:
# # # test code
# file= "1008_Session_1_ET_2_data_interpolated.csv"
# data = pd.read_csv(os.path.join(data_savepath_interpolated, file))
# data.head()


In [8]:
# pd.set_option('display.max_columns', None)

# # # Assuming 'data' is your DataFrame
# data_head = data.tail(15)

# # Display head horizontally
# with pd.option_context('display.max_rows', None, 'display.width', 1000):
#     display(data_head)
# #     display(data.iloc[100:170])

# Gaze Processing


In [6]:
# This will be used to select which version to use
# 1 == MAD_woBig # this is the version we should use (data-driven)
# 2 == 10_sec
define_intervals = 1

# Caluclate angles & velocities

## Calculate the eye-tracking veloctiy

In [6]:
# p.s.
#
# v_gaze_vec(t) = [Xhpoo(t) - Xhpoo(t-1), Yhpoo(t) - Yhpoo(t-1), Zhpoo(t) - Zhpoo(t-1)]
#
# Now we project this velocity onto the plane orthogonal to the viewing
# direction at time t and determine the length. For simplicity we assume
# that all vertical distances are small and keep the plane vertically
# oriented.
#
# v_gaze_inplane(t) = ||v_gaze_vec(t) - (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))||
#
# where <> indicates the scalar product
# gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in
# world coordinates.
# v_gaze_inplane is a scalar indicating the velocity in world coordinates
# at the location that is gazed at orthogonal to the gaze axis.
# || ... || indicates the euclidian norm
#
# w_gaze(t) = arctan2(distance(subject(i), hpoo(t)), v_gaze_inplane)
#
# distance(subject(i), hpoo(t)) could be equivalently written as ||
# subject_vec(t) - hpoo_vec(t)||
#
# w_gaze is the instantaneous angular velocity of the eye movement and
# what we want to know.

In [7]:
# note this code calculates the velocity between the rows gaze coordinates
# note 2 the velocity calculation takes the time difference between rows into account, so it should not be affected by the 
# varying sampling rate

def velocity_calc(file, data):

    time = data["timeStampRS"].tolist()

    # get individual coordinates
    # position
    Xcorr_position = data["eyePositionCombinedWorld_x"].tolist()
    Ycorr_position = data["eyePositionCombinedWorld_y"].tolist()
    Zcorr_position = data["eyePositionCombinedWorld_z"].tolist()
    subj = list(zip(Xcorr_position, Ycorr_position, Zcorr_position))

    # hpoo
    hpooX = data["processedCollider_hitPointOnObject_x"].tolist()
    hpooY = data["processedCollider_hitPointOnObject_y"].tolist()
    hpooZ = data["processedCollider_hitPointOnObject_z"].tolist()
   
    hpoo = list(zip(hpooX, hpooY, hpooZ))

    # v_gaze_vec: get difference in hpoo
    v_vX = pd.DataFrame(hpooX).apply(lambda x: x.diff())[0].tolist()
    v_vY = pd.DataFrame(hpooY).apply(lambda x: x.diff())[0].tolist()
    v_vZ = pd.DataFrame(hpooZ).apply(lambda x: x.diff())[0].tolist()

    # get difference in time:
    ts = pd.DataFrame(time).apply(lambda x: x.diff())[0].tolist()

    v_gaze_vec = list(zip(v_vX, v_vY, v_vZ))

    # gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in world coordinates
    g_vec = [np.array(hpoo[v] - np.array(subj[v])) for v in range(len(subj))]
    gaze_vec = [np.array(v) / np.linalg.norm(np.array(v)) for v in g_vec]
    # display(np.linalg.norm(gaze_vec[1]))

    # v_gaze_inplane: is a scalar indicating the velocity in world coordinates at the location that is gazed at orthogonal to the gaze axis.
    # v_gaze_inplane(t) = ||v_gaze_vec(t) - (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))||:
    # z1 = (<v_gaze_vec(t), gaze_vec(t)>)
    z1 = [
        np.array(v_gaze_vec[t]).dot(np.array(gaze_vec[t]))
        for t in range(len(v_vX))
    ]

    # z = (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))
    z = [z1[t] * np.array(gaze_vec[t]) for t in range(len(v_vX))]

    # ||v_gaze_vec(t) - z||
    v_gaze_inplane = [
        np.linalg.norm((np.array(v_gaze_vec[t]) - z[t]).tolist())
        for t in range(len(v_gaze_vec))
    ]
    # w_gaze(t) = arctan2(||subject_vec(t) - hpoo_vec(t)||, v_gaze_inplane)

    # sub_hpoo = ||subject_vec(t) - hpoo_vec(t)||
    sub_hpoo = [
        np.linalg.norm(np.array(subj[t]) - np.array(hpoo[t]))
        for t in range(len(hpoo))
    ]

    # arctan2(v_gaze_inplane, sub_hpoo)
    w_gaze = np.arctan2(v_gaze_inplane, sub_hpoo).tolist()

    # turn angle of radians into degrees
    w_gaze = [(w / ts[idx] * 180 / math.pi) for idx, w in enumerate(w_gaze)]
    # w_gaze = [(w*180/math.pi) for idx,w in enumerate(w_gaze)]


    
    # save df --> to facilitate runtime, save data only in last processing function
    data["combined_vel"] = w_gaze
    


    return data
        

In [8]:
## add new bool variable to identify nan values. Since the first value of a data segment will not have a velocity
# a new variable is needed to successfully index valid data and avoid indexing inconsistencies


def id_valid_data(file, data):
    
    
    nanVels = np.isnan(data["combined_vel"])
    data["validData"] = ~nanVels
    
    return data

In [9]:
### MAXIMUM VELOCITY THRESHOLD (1000deg/sec)
# note: this function just replaces all velocity values exceeding the max threshold of 1000deg/sec with 1000.0. 
# this part should not be affected by the varying sampling rate
 
def max_vel_threshold(file, data):
    
    # max_vel threshold
    max_vel = 1000.0

    combined_vel = data["combined_vel"].tolist()
    c_v = [max_vel if cv > max_vel else cv for cv in combined_vel]
    data["combined_vel"] = c_v
    
    
    return data
        

# Smooth the Data (Savitzky–Golay)

In [10]:
# Taken from NYSTRÖM AND HOLMQVIST, 2010
# Filter: Savitzky–Golay (sgolay in MATLAB)
# Filter order: 2
# Filter length: 2* min saccade duration Peak

# this might need adjustment, since the sampling rate is taken into consideration for savgol_length...

# also please note: the savitzky_golay filter does not converge if the file starts or ends with
# nan values! The code checks for this case and removes the nan rows for the filtering. 

# also, it cannot handle the remaining nan segments well. It always removes the data from the before and after the 
# nan-data segment by replacing it with NaN values - causing additional data loss and inconsistencies during indexing later
# therefore, the data is split into non-nan value segments and each segment is processed individually to avoid the additional 
# data loss

def savitzky_golay(file, data): 

    # define parameters
    savgol_length = 3  # 0.01 * 90 #window size = int(savgol_length * sr); sr = 90; remodnav = 0.019 -->  int(savgol_length * sr) not possible with our SR
    savgol_polyord = 2  # polynomial order


    combined_vel = data["combined_vel"].values
    
    # identify the non-nan value segments
    nans = ~data["validData"]
    non_nan_segments = np.where(~nans)[0]
    
    if len(non_nan_segments) > 0:
        segment_starts = np.split(non_nan_segments, np.where(np.diff(non_nan_segments) != 1)[0] + 1)
        
        # apply the filter to all segments
        for segment in segment_starts:   
            if len(segment) > savgol_length:
                
                # Apply the Savitzky-Golay filter to the non-NaN segment   
                filtered_segment = scipy.signal.savgol_filter(combined_vel[segment], savgol_length, savgol_polyord)
                # Store the filtered segment back into the result array            
                combined_vel[segment] = filtered_segment
                
    # save data
    data["combined_vel"] = combined_vel
    
    return data
        





# Function at_mad

In [11]:
# Function: calculate MAD saccade
def at_mad(angular_vel, th_0=200):
    # defines the saccade threshold (code from Ashima)
    threshs = []
    while True:
        # take th_0
        threshs.append(th_0)
        
        # get all angles smaller than this
        angular_vel = angular_vel[angular_vel < th_0]
        angular_vel = angular_vel[angular_vel != 0]
               
        # check if angular_vel contains any values to avoid warnings in the calculation of the median ect. (modification by Jasmin)
        if(angular_vel.size == 0):
            th_1 = np.nan
        
        else:
            # MAD:
            # take the median of all angles smaller than th_0 
            median = np.median(angular_vel)            
            # substract the median value
            diff = np.sqrt((angular_vel - median) ** 2)
            # get the median of these values
            med_abs_deviation = np.median(diff)
            # calcualte the next threshold with the median
            # 1.486 used when assuming a normal distribution
            th_1 = median + 3 * 1.486 * med_abs_deviation
                    # if the thresholds are too different, redo the while loop
        if abs(th_0 - th_1) > 1:
            th_0 = th_1
        # else, set the final threshold to the current one, break the while loop and return values
        else:
            saccade_thresh = th_1
            threshs.append(saccade_thresh)
            break
    return saccade_thresh, threshs

In [13]:
# Function: calculate MAD saccade
def at_mad_normalized(angular_vel, th_0=200):
    # defines the saccade threshold (code from Ashima)
    threshs = []
    
    
    while True:
        threshs.append(th_0)
        
        # Normalize angular velocity values
        max_angular_vel = np.max(np.abs(angular_vel))
        normalized_angular_vel = angular_vel / max_angular_vel
        
        # Filter normalized angular velocity
        normalized_angular_vel = normalized_angular_vel[normalized_angular_vel < th_0]
        normalized_angular_vel = normalized_angular_vel[normalized_angular_vel != 0]
        
        print('length angular vel', len(angular_vel))
        
        if normalized_angular_vel.size == 0:
            th_1 = np.nan
        else:
             # MAD:
            # Calculate median and MAD on normalized data
            median = np.median(normalized_angular_vel)
            print('median',median)
            diff = np.abs(normalized_angular_vel - median)
            med_abs_deviation = np.median(diff)
            # Calculate threshold based on normalized values
            th_1 = median + 3 * 1.486 * med_abs_deviation
            
            # Scale threshold back to original units
            th_1 = th_1 * max_angular_vel
        
        print(th_1)
        print( abs(th_0 - th_1))
        
        if abs(th_0 - th_1) > 1:
            th_0 = th_1
        else:
            saccade_thresh = th_1
            threshs.append(saccade_thresh)
            break
    return saccade_thresh, threshs



# Calculate the data segmenation intervals

## Exclude big saccs

In [14]:
# Calcualte the threshold across the entire dataset

# determining a critical velocity on a median-filtered (median filter length) time series comprising
# the full duration of a recording. All such periods of consecutive above-threshold velocities are
# weighted by the sum of these velocities. Boundaries of time series chunks are determined by selecting
# such events sequentially (starting with the largest sums) until a maximum average frequency across
# the whole time series is reached resulting chunks represent data intervals between saccades of maximum
# magnitude in the respective data.


def calc_mad_wobig_interval(file, data):
    
    # get time
    time = data["timeStampRS"].tolist()

    # run MAD saccade on the entire data
    saccade_th, thres = at_mad(data["combined_vel"])

    # detect all periods above the calculated threshold
    fast_sacc = [
        ti if ti >= saccade_th else np.nan
        for ti in data["combined_vel"].tolist()
    ]
    
#     test = pd.DataFrame(data["combined_vel"])
#     test["fast_sacc"] = fast_sacc
#     print(test[:15])
    
#     interv = [np.nan] * len(time)  # rem
#     interv[0] = 600  # rem
#     total_v = [np.nan] * len(time)  # rem
    start = [time[0]]  # onset time
    start_idx = [0]  # onset idx
    end = []  # offset time
    end_idx = []  # offset index
    total_vels = []  # save the velocities to sort the fast_sacc

    # to add and update for each interval
    starts = []  # save the indicies in
    vels = []  # save the velocities in
    cnt = True  # every time it will be set to true, add the velocity
    curr_end = 0 ############################################################## to fix code --> assign curr_end to 0 to catch the case that fast_sacc[0] is not NaN

    for v in range(1, (len(fast_sacc) - 1)): ################################## to fix code --> range(0, (lenfast_sacc )-1) --> why would it not start with the first data point anyway???
        # if you start an event higher than the threshold:
        
        # if current line is NaN and next line is not NaN
        if pd.isnull(fast_sacc[v]) and not pd.isnull(fast_sacc[v + 1]):
            cnt = True
            curr_end = v + 1
        # if current line is not NaN and next line is NaN
        elif not pd.isnull(fast_sacc[v]) and pd.isnull(fast_sacc[v + 1]) and curr_end > 0:
                    
#             print("enter if - v= ", v, "fast_sacc[v]",fast_sacc[v], "fast_sacc[v + 1]", fast_sacc[v + 1])
            # vels:
            total_vels.append(np.nansum(vels))
#             print(total_v[starts[vels.index(np.nanmax(vels))]])
#             print("here")
#             total_v[starts[vels.index(np.nanmax(vels))]] = np.nansum(
#                 vels
#             )  # rem
            # start
            start_idx.append(v + 1)
            start.append(time[v + 1])
            # end
            end_idx.append(curr_end)
            end.append(time[curr_end])
#             interv[starts[vels.index(np.nanmax(vels))] + 1] = 600  # rem
            # interv[starts[vels.index(np.nanmax(vels))]] = 400 #rem
            # reset:
            cnt = False
            starts = []  # save the indicies in
            vels = []  # save the velocities in
        # if you are currently looking at intervals higher than the theshold: add them to the lists
        if (
            cnt == True
        ):  # and not pd.isnull(fast_sacc[v]) and pd.isnull(fast_sacc[v+1]):
            starts.append(v + 1)  # get the index
            vels.append(fast_sacc[v + 1])  # get the velocities

    end.append(time[-1])
    end_idx.append(len(fast_sacc))
    total_vels.append(np.nansum(vels))
    # interv[-1] = 400 #rem

    # order the data
    int_data = pd.DataFrame(
        list(zip(start, end, start_idx, end_idx, total_vels)),
        columns=["start", "end", "start_idx", "end_idx", "total_vels"],
    )

    # only get the big saccades
    max_initial_saccade_freq = 0.5
    # order the indicies according to the frequencies
    int_data = int_data.sort_values("total_vels", ascending=False)

    # if still smaller than max_initial_saccade_freq, keep adding nr to list, else, add 0
    new_start = [int_data["start"][0]]
    to_keep = [1]
    for st in int_data["start"][1:].tolist():
        new_start.append(st)
        new_start.sort()
        new_dist = [
            new_start[i + 1] - new_start[i] for i in range(len(new_start) - 1)
        ]
        if sum(new_dist) / len(new_dist) > max_initial_saccade_freq:
            to_keep = to_keep + [1]
        else:
            break

    to_keep = to_keep + [0] * (len(start) - len(to_keep))
    int_data["to_keep"] = to_keep
    int_data = int_data.sort_values("start_idx")

    # so we can make sure we will cover the entire duration
    to_keep = int_data["to_keep"].tolist()
    to_keep[0] = 1
    to_keep[-1] = 1
    int_data["to_keep"] = to_keep

    # now only save the list-elements we want to keep
    adjusted = True
    for v, val in enumerate(int_data["to_keep"].tolist()):
        if val == 0 and adjusted:
            curr_idx = v
            adjusted = False
        elif val == 1 and not adjusted:
            int_data.loc[v, "start"] = int_data["start"][curr_idx]
            int_data.loc[v, "start_idx"] = int_data["start_idx"][curr_idx]
            adjusted = True

    # delete all rows with 0
    int_data = int_data[int_data["to_keep"] == 1].reset_index()
    int_data = int_data.drop(columns=["to_keep"])
 


    return data, int_data

# Calculate MAD Saccade

## Get the thresholds for each interval

In [16]:
# go through the caclulated intervals, run at_mad on each of these intervals and save the thresholds

def mad_saccade(file, data, int_data):

    combined_vel = data["combined_vel"]
        
    # to shorten the slicing in the next for loop
    start_idx = int_data["start_idx"].tolist()
    end_idx = int_data["end_idx"].tolist()

    # to add the final thresholds to for each segement
    scct = []
    for s, srt in enumerate(int_data["start"].tolist()):            

        # get the slice of the combined velocity
        angular_vel = combined_vel[start_idx[s] : end_idx[s]]
        # use the at_mad function to caluclate the threshold
        saccade_th, thres = at_mad(angular_vel)
        if np.isnan(saccade_th):
            scct.append(thres[0])
        else:
            # add it to scct
            scct.append(saccade_th)

    # add it to int_data and save
    int_data["thresh"] = scct

#     display(len(scct))
    int_data = pd.DataFrame(int_data)

    
    # save intervals 
#     # 1 == MAD_woBig
    if define_intervals == 1:
        # save interval
        saveName = file.replace("data_resampled.csv","interval_mad_wobig.csv")
        int_data.to_csv(os.path.join(data_savepath_intervals, saveName), index = False)
    # 2 == 10_sec
    else:
        print("why are you using the 10 sec interval, it is not fully adapted in this script")
    
    
    return data, int_data

## Add the threshold to data df

In [17]:
# go through the caclulated intervals, run at_mad on each of these intervals and save the thresholds

def add_threshold(file, data, int_data):

    time = data.index.tolist()
    start_idx = int_data["start_idx"].tolist()
    end_idx = int_data["end_idx"].tolist()
    thr = int_data["thresh"].tolist()

    # go through all time intervals
    thresh = [0.0] * len(time)
    for s, srt in enumerate(int_data["start"].tolist()):
        # repeat the threshold as often as the time interval is long
        thresh = (
            thresh[: start_idx[s]]
            + [thr[s]] * len(time[start_idx[s] : end_idx[s]])
            + thresh[end_idx[s] :]
        )

    # add the two lists (ht & et) to for_eye df
    data["thresh"] = thresh

    # save for_eye df
    data = pd.DataFrame(data)


    return data, int_data

## Define Fixations

In [18]:
def define_fixations(file,data,int_data):
    
    start = int_data["start_idx"].tolist()
    end = int_data["end_idx"].tolist()
    thres = int_data["thresh"].tolist()
    combined_vel = data["combined_vel"].tolist()

    # define list where the fixations will be added too
    is_fix = [np.nan] * len(combined_vel)

    for i in range(len(start)):
        av = combined_vel[start[i] : end[i]]
        # go through combined velocity and save all that are smaller than the threshold
        fix = [ti if ti < thres[i] else np.nan for ti in av]
        is_fix[start[i] : end[i]] = fix

    # save
    data["isFix"] = is_fix

    # save data
    data = pd.DataFrame(data)


    return data, int_data

## Correct small saccades and add gaze length + event onset/offset to list

In [51]:
def identify_first_event_index(file, data):
    

    min_sacc_dur = 0.02  # min sacc duration
    min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)

    index = data.index.tolist()  # index of df for easier use
    start_time = data.iloc[1]["timeStampRS"]  # update for each change
    start_idx = index[1]  # will be updated each event and used to add to the lists
    
    first_event_idx = start_idx  # identify when the first event starts that fulfulls the min sacc or min gaze duration. 



    # starting with a sacc
    if math.isnan(data.loc[start_idx]["isFix"]):
        event = 0  # == sacc
    # starting with a gaze
    else:
        event = 1  # == gaze
        

    for idx in index[index.index(start_idx) : -1]:
        
        current_event_length = data.iloc[idx+1]["timeStampRS"] - start_time 
        
        if event == 0: # if the current event is a saccade
            
            if current_event_length < min_sacc_dur: # check, if the saccade is not yet long enough
                
                if not math.isnan(data.loc[idx+1]["isFix"]): # if the next line is not a saccade, switch event
                    event = 1 
                    start_idx = idx + 1
                    start_time = data.iloc[idx+1]["timeStampRS"]
            else:
                
                first_event_idx = start_idx
                break # in case the min saccade length has been reached, break the for loop and store the index
                
        else: # if the current event is a gaze
             
            if current_event_length < min_gaze_dur: # check, if the saccade is not yet long enough
                
                if math.isnan(data.loc[idx+1]["isFix"]): # if the next line is not a gaze, switch event
                    event = 0
                    start_idx = idx + 1
                    start_time = data.iloc[idx+1]["timeStampRS"]
            else:
                
                first_event_idx = start_idx
                break # in case the min saccade length has been reached, break the for loop and store the index
        
                
        
    return first_event_idx
        
    

In [52]:
##### CODE WORKING SO FAR
def correct_small_events(file, data, first_event_idx):
    """
    Correct is an event is too small.

    Parameters:
    """


    min_sacc_dur = 0.02  # min sacc duration
    min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)

    index = data.index.tolist()  # index of df for easier use
    start_time = data.iloc[first_event_idx]["timeStampRS"]  # update for each change
    start_idx = index[first_event_idx]  # will be updated each event and used to add to the lists

    # to save:
    isFix = []
    combined_vel = []
    
    

#     # if the first sample does not have any data
#     if math.isnan(data.iloc[0]["combined_vel"]) and not math.isnan(
#         data.iloc[1]["combined_vel"]
#     ):
#         start_time = data.iloc[1]["timeStampRS"]  # update for each change
#         start_idx = index[
#             1
#         ]  # will be updated each event and used to add to the lists

    # iterate index to identify the number of necessary empty rows to add
    number_rm_rows = 0
    for nr, i in enumerate(index):
        
        if i == first_event_idx:
            number_rm_rows = nr
            isFix = [np.nan]*nr
            combined_vel = [np.nan]*nr
            break
        

    # starting with a sacc
    if math.isnan(data.loc[start_idx]["isFix"]):
        event = 0  # == sacc
    # starting with a gaze
    else:
        event = 1  # == gaze

    # hits progress bar
    index_pbar = tqdm(
        iterable=index[index.index(start_idx) : -1],
        desc=f"⌚ timestamps from file {file}",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )

    # display(for_eye[['time','isFix']])
    # go through the list:
    for idx in index_pbar:
        curr_line = data.loc[idx]
        next_line = data.loc[idx + 1]

        # gaze (--> sacc): now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):
            
            # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
            if event == 1 and next_line.timeStampRS - start_time < min_gaze_dur:
                isFix = (
                    isFix + data.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
                
            # elif current event to small and we are in big saccade, change isFix, change combined_vel
            elif event == 0 and next_line.timeStampRS - start_time < min_gaze_dur:
                isFix = isFix + [np.nan] * (idx + 1 - start_idx)
                combined_vel = combined_vel + [np.nan] * (idx + 1 - start_idx)
                
            # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
            elif next_line.timeStampRS - start_time >= min_gaze_dur:
                isFix = (
                    isFix + data.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + data.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 1  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = data.loc[idx + 1]["timeStampRS"]

        # sacc (--> gaze): now sacc, next one is gaze
        elif math.isnan(curr_line.isFix) and not math.isnan(next_line.isFix):
            
            # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
            if event == 0 and next_line.timeStampRS - start_time < min_sacc_dur:
                isFix = (
                    isFix + data.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
                
            # elif current event to small but we are in big gaze, change isFix, change combined_vel
            elif event == 1 and next_line.timeStampRS - start_time < min_sacc_dur:
                isFix = (
                    isFix
                    + data.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # change isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
                
            # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
            elif next_line.timeStampRS - start_time >= min_sacc_dur:
                isFix = (
                    isFix + data.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + data.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 0  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = data.loc[idx + 1]["timeStampRS"]

        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
                if (
                    event == 1
                    and next_line.timeStampRS + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = (
                        isFix
                        + data.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small and we are in big saccade, change isFix, change combined_vel
                elif (
                    event == 0
                    and next_line.timeStampRS + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = isFix + [np.nan] * (idx + 2 - start_idx)
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )
                # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
                elif next_line.timeStampRS + 0.011 - start_time >= min_gaze_dur:
                    isFix = (
                        isFix
                        + data.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + data.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel
            # sacc:
            elif math.isnan(next_line.isFix):
                # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
                if (
                    event == 0
                    and next_line.timeStampRS + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + data.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small but we are in big gaze, change isFix, change combined_vel
                elif (
                    event == 1
                    and next_line.timeStampRS + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + data.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # change isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
                elif next_line.timeStampRS + 0.011 - start_time >= min_sacc_dur:
                    isFix = (
                        isFix
                        + data.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + data.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel

    # save everything:
    data["isFix"] = isFix
    data["corrected_vel"] = combined_vel
    # save data
    data = pd.DataFrame(data)

  
    return data, number_rm_rows



## Define events, duration etc. 

In [53]:
def get_events_len_dist(file, data, first_event_idx, nr_rm_rows):
    """
    assign event values and other variables, e.g. length, distance etc.

    Parameters:
        uid (str): Participant identifier.
    """
    
    min_sacc_dur = 0.02  # min sacc duration
    min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)

    time = data["timeStampRS"].tolist()


    ########## EVENTS, LENGTH, AVG DISTANCE, NAME OF OBJECT ##########
    index = data.index.tolist()  # index of df for easier use

    events = [np.nan] * len(data)  # sacc begin == 1, sacc end == -1; 
                                   # gaze begin == 2, gaze end == -2, 
                                   # invalid data begin == 3, invalid data end == -3
                                   # invalid starting samples begin == 4, invalid starting samples end == -4
    
    # preallocate space for better performance
    length = [np.nan] * len(data) 
    dist = [np.nan] * len(data)   # to save the distance to the hitpoint at each timestamps
    avg_dist = ([np.nan] * len(data))  # to save the average distance of collider(s) during event
    names = [np.nan] * len(data)   # to save the name of the current gaze

    ## NH
    distNH = [np.nan] * len(data)   # to save the distance to the hitpoint at each timestamps
    avg_distNH = ([np.nan] * len(data) )  # to save the average distance of collider(s) during event
    namesNH = [np.nan] * len(data)   # to save the name of the current gaze
    ###



    # check where the first event starts and fill all other values for the invalid start section. 
    
    start_idx = first_event_idx  # will be updated each event and used to add to the lists
    
    length[0:nr_rm_rows] = [data.loc[first_event_idx]["timeStampRS"]] *nr_rm_rows
   

    # event naming
    events[0] = 4
      
    if not(nr_rm_rows-1 == 0):
        events[index[nr_rm_rows-1]] = -4

    if math.isnan(data.iloc[start_idx]["isFix"]):
        events[start_idx] = 1
    else:
        events[start_idx] = 2
               


    start_time = data.loc[start_idx]["timeStampRS"].tolist()

    # hits progress bar
    index_pbar = tqdm(
        iterable=index[index.index(start_idx) : -1],
        desc=f"⌚ timestamps from file {file}",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )


    # go through the list:
    for idx in index_pbar:
        curr_line = data.loc[idx]
        next_line = data.loc[idx + 1]

        # distance:
        hpoo = np.array(
            [curr_line["processedCollider_hitPointOnObject_x"], 
             curr_line["processedCollider_hitPointOnObject_y"], 
             curr_line["processedCollider_hitPointOnObject_z"]]
        )  # hitpoints on object

                    # NH hpoo
        hpooNH = np.array(
            [curr_line["processedCollider_NH_hitPointOnObject_x"], 
             curr_line["processedCollider_NH_hitPointOnObject_y"], 
             curr_line["processedCollider_NH_hitPointOnObject_z"]]
        )  


        coord_orig = np.array(
            [
                curr_line["eyePositionCombinedWorld_x"],                           
                curr_line["eyePositionCombinedWorld_y"],                          
                curr_line["eyePositionCombinedWorld_z"],
            ]
        )  # position of eyes
        dist[idx] = np.linalg.norm(hpoo - coord_orig)  # calculate to distance at this timpoint

        distNH[idx] = np.linalg.norm(hpooNH - coord_orig)  # calculate to distance at this timpoint


        # gaze --> sacc: now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):

            # get name:
            res = dict(
                Counter(data.loc[start_idx:idx, "processedCollider_name"].values.tolist())
            )
            # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
            try:
                max_keys = max((key for key in res.keys() if isinstance(key, str)), key=(lambda new_k: res[new_k]))
            except ValueError:
                max_keys = np.nan
            names[start_idx:idx] = [max_keys] * (idx + 1 - start_idx)
            res = []

            ############# same for NH names

            resNH = dict(
                Counter(data.loc[start_idx:idx, "processedCollider_NH_name"].values.tolist())
            )
            # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
            try:
                max_keysNH = max((key for key in resNH.keys() if isinstance(key, str)), key=(lambda new_k: resNH[new_k]))
            except ValueError:
                max_keysNH = np.nan
            namesNH[start_idx:idx] = [max_keysNH] * (idx + 1 - start_idx)
            resNH = []


            #######################

            # length, distance, events
            length[start_idx:idx] = [next_line.timeStampRS - start_time] * (
                idx + 1 - start_idx
            )  # length of event

            avg_dist[start_idx:idx] = [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event

            ## NH avg_dist
            avg_distNH[start_idx:idx] = [
                np.nanmean(distNH[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event

            events[index.index(idx)] = -2  # end of gaze

            # check if next line is a fixation or a invalidData segment
            if next_line.validData :
                events[index.index(idx) + 1] = 1  # beginning of sacc
            else:
                events[index.index(idx) + 1] = 3 # beginn of invalid data segment

            # new idx
            start_time = curr_line.timeStampRS
            start_idx = idx + 1


        # sacc --> not: now sacc, next one is not a saccade
        elif (math.isnan(curr_line.isFix) and curr_line.validData) and ((not math.isnan(next_line.isFix)) | (not next_line.validData) ):

            # get name:
    #         res = dict(
    #             Counter(data.loc[start_idx:idx, "processedCollider_name"].values.tolist())
    #         )
    #         # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
    #         try:
    #             max_keys = max((key for key in res.keys() if isinstance(key, str)), key=(lambda new_k: res[new_k]))

    #         except ValueError:
    #             max_keys = np.nan
            names[start_idx:idx] = data.loc[start_idx:idx, "processedCollider_name"].values.tolist()
            res = []

            ############# same for NH names

    #         resNH = dict(
    #             Counter(data.loc[start_idx:idx, "processedCollider_NH_name"].values.tolist())
    #         )
    #         # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
    #         try:
    #             max_keysNH = max((key for key in resNH.keys() if isinstance(key, str)), key=(lambda new_k: resNH[new_k]))

    #         except ValueError:
    #             max_keysNH = np.nan
            namesNH[start_idx:idx] = data.loc[start_idx:idx, "processedCollider_NH_name"].values.tolist()
            resNH = []


            ###############################


            # length, distance, events
            length[start_idx:idx] = [next_line.timeStampRS - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist[start_idx:idx] = [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event

            ## NH
            avg_distNH[start_idx:idx] = [
                np.nanmean(distNH[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event

            # check if next line is a fixation or an invalidData segment
            if not math.isnan(next_line.isFix):
                events[index.index(idx) + 1] = 2  # beginning of gaze
            else:
                events[index.index(idx) + 1] = 3 # beginn of invalid Data segment

            ###### TO CHANGE POSSIBLY
            # only ends the saccade, if it is longer than 1 sample/row long
            if events[index.index(idx)] != 1:
                events[index.index(idx)] = -1  # end of sacc

            # new idx
            start_time = curr_line.timeStampRS
            start_idx = idx + 1

         # invalidData --> not: now invalid Data, next one is not
        elif (math.isnan(curr_line.isFix) and (not curr_line.validData)) and ((not math.isnan(next_line.isFix)) | next_line.validData ):

            # get name:
    #         res = dict(
    #             Counter(data.loc[start_idx:idx, "processedCollider_name"].values.tolist())
    #         )
    #         # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
    #         try:
    #             max_keys = max((key for key in res.keys() if isinstance(key, str)), key=(lambda new_k: res[new_k]))

    #         except ValueError:
    #             max_keys = np.nan
            names[start_idx:idx] = [np.nan] * (idx + 1 - start_idx)
            res = []

            ############# same for NH names

    #         resNH = dict(
    #             Counter(data.loc[start_idx:idx, "processedCollider_NH_name"].values.tolist())
    #         )
    #         # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
    #         try:
    #             max_keysNH = max((key for key in resNH.keys() if isinstance(key, str)), key=(lambda new_k: resNH[new_k]))

    #         except ValueError:
    #             max_keysNH = np.nan
            namesNH[start_idx:idx] = [np.nan] * (idx + 1 - start_idx)
            resNH = []


            ###############################


            # length, distance, events
            length[start_idx:idx] =  [next_line.timeStampRS - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist[start_idx:idx] = [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event

            ## NH
            avg_distNH[start_idx:idx] = [
                np.nanmean(distNH[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event

            # check if next line is a fixation or an invalidData segment
            if not math.isnan(next_line.isFix):
                events[index.index(idx) + 1] = 2  # beginning of gaze
            else:
                events[index.index(idx) + 1] = 1 # beginn of saccade

            ###### TO CHANGE POSSIBLY
            # only ends the saccade, if it is longer than 1 sample/row long
            if events[index.index(idx)] != 3:
                events[index.index(idx)] = -3  # end of sacc

            # new idx
            start_time = curr_line.timeStampRS
            start_idx = idx + 1



        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                if not events[-1] == 2:
                    events[-1] = -2  # end of gaze
            # sacc:
            elif math.isnan(next_line.isFix) and next_line.validData:
                if not events[-1] == 1:
                    events[-1] = -1  # end of sacc
            elif math.isnan(next_line.isFix) and not next_line.validData:
                if not events[-1] == 3:
                    events[-1] = -3 # end of invalid data 



            length[start_idx:] = [next_line.timeStampRS - start_time] * (
                idx + 2 - start_idx
            )  # length of event
            # distance
            avg_dist[start_idx:] = [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 2 - start_idx
            )  # average distance to collider(s) during event

            ## NH
            avg_distNH[start_idx:] = [
                np.nanmean(distNH  [index.index(start_idx) :])
            ] * (
                idx + 2 - start_idx
            )  # average distance to collider(s) during event

             # hitpoints on object

            hpoo = np.array(
                [next_line["processedCollider_hitPointOnObject_x"], 
                 next_line["processedCollider_hitPointOnObject_y"], 
                 next_line["processedCollider_hitPointOnObject_z"]]
            )  
            # NH hpoo
            hpooNH = np.array(
                [next_line["processedCollider_NH_hitPointOnObject_x"], 
                 next_line["processedCollider_NH_hitPointOnObject_y"], 
                 next_line["processedCollider_NH_hitPointOnObject_z"]]
            )  

            coord_orig = np.array(
                [
                    next_line["eyePositionCombinedWorld_x"],
                    next_line["eyePositionCombinedWorld_y"],
                    next_line["eyePositionCombinedWorld_z"],
                ]
            )  # position of eyes
            dist[idx+1] = [
                np.linalg.norm(hpoo - coord_orig)
            ]  # calculate to distance at this timpoint

            # NH dist
            distNH[idx+1] = [
                np.linalg.norm(hpooNH - coord_orig)
            ]  # calculate to distance at this timpoint


            # names
            # if it is a gaze
            if events[-1] == 2 or events[-1] == -2:
                res = dict(
                    Counter(data.loc[start_idx:, "processedCollider_name"].values.tolist())
                )
                # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
                try:
                    max_keys = max((key for key in res.keys() if isinstance(key, str)), key=(lambda new_k: res[new_k]))
                except ValueError:
                    max_keys = np.nan
                names[start_idx:] = [max_keys] * (idx + 2 - start_idx)
                res = []

                ############# same for NH names

                resNH = dict(
                    Counter(data.loc[start_idx:, "processedCollider_NH_name"].values.tolist())
                )
                # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
                try:
                    max_keysNH = max((key for key in resNH.keys() if isinstance(key, str)), key=(lambda new_k: resNH[new_k]))
                except ValueError:
                    max_keysNH = np.nan
                namesNH[start_idx:] = [max_keysNH] * (idx + 2 - start_idx)
                resNH = []
            # if it is a saccade
            elif events[-1] == 1 or events[-1] == -1:
                names[start_idx:] = data.loc[start_idx:, "processedCollider_name"].values.tolist()
                namesNH[start_idx:] = data.loc[start_idx:, "processedCollider_NH_name"].values.tolist()

            # if it ends with a invalid segment
            elif events[-1] == 3 or events[-1] == -3:
                names[start_idx:] = [np.nan] * (idx + 2 - start_idx)
                namesNH[start_idx:] = [np.nan] * (idx + 2 - start_idx)

            #############################

        # display(len(names))
        # display(len(avg_dist))
    # save everything:
    data["events"] = events
    data["length"] = length
    data["distance"] = dist
    data["distance_NH"] = distNH
    data["avg_dist"] = avg_dist
    data["avg_dist_NH"] = avg_distNH
    data["names"] = names
    data["namesNH"] = namesNH

    # save data
    data = pd.DataFrame(data)


    return data



## Adjust the distance

In [56]:
# Change average distance to correct for the potential of other events
# so distance and avg_dist
# plot average duration of gazes and saccades

def adjust_distance(file, data):

    # total lists:
    all_dist = []
    avg_dist = []
    # updated after each gaze
    dist = []
    hon_pos = []
    
    
    dur_gaze = False
    
    # during gaze:
    # go through the list:
    for g, gz in enumerate(data["events"]):
        curr_line = data.loc[g]
        if gz == 2.0 or gz == 1.0:
            dur_gaze = True
            # get the gazed at object
            curr_gaze = curr_line.names
        # if you are currently in a gaze:
        if dur_gaze:
            # if you are currently having the correct element, add the position
            if curr_line.processedCollider_name == curr_gaze:
                hon_pos = hon_pos + [
                    [
                        curr_line["processedCollider_hitPointOnObject_x"], 
                        curr_line["processedCollider_hitPointOnObject_y"], 
                        curr_line["processedCollider_hitPointOnObject_z"]
                    ]
                ]
            dist = dist + [
                np.array(
                    [
                        curr_line["eyePositionCombinedWorld_x"],
                        curr_line["eyePositionCombinedWorld_y"],
                        curr_line["eyePositionCombinedWorld_z"],
                    ]
                )
            ]

        # once the gaze is over, take the avg_dist
        if gz == -2.0 or gz == -1.0:
            hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
            # calculate to distance at this timpoint
            dist = [
                np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))
            ]
            all_dist = all_dist + dist
            # average distance during the gaze event
            avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

            # reset everything:
            dist = []
            hon_pos = []
            dur_gaze = False

        # if there are parts that are neither gaze nor saccade:
        if (
            (not dur_gaze)
            and (gz not in [2.0, 1.0])
            and (len(all_dist) + len(dist) != g + 1)
        ):
            all_dist = all_dist + [np.nan]
            avg_dist = avg_dist + [np.nan]

        if len(all_dist) + len(dist) != g + 1:
            display(g)

    if dur_gaze:
        hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
        # calculate to distance at this timpoint
        dist = [np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))]
        all_dist = all_dist + dist
        # average distance during the gaze event
        avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

    # add them to for_eye
    data["distance"] = all_dist
    data["avg_dist"] = avg_dist

    # save data
    data = pd.DataFrame(data)
    
    
    # save df:
    # 1 == MAD_woBig
    
    if define_intervals == 1:
        # save interval
        saveName = file.replace("data_resampled.csv","data_correTS_mad_wobig.csv")
        data.to_csv(os.path.join(data_savepath_intermediateP, saveName), index = False)
        
    # 2 == 10_sec
    else:
        print("why are you using the 10 sec interval, it is not fully adapted in this script")

    return data

In [57]:
# Same function as above just for NH variables

# Change average distance to correct for the potential of other events
# so distance and avg_dist
# plot average duration of gazes and saccades

def adjust_distance_NH(file, data):

    # total lists:
    all_distNH = []
    avg_distNH = []
    # updated after each gaze
    distNH = []
    hon_posNH = []
    
    dur_gazeNH = False
    
    # during gaze:
    # go through the list:
    for g, gz in enumerate(data["events"]):
        curr_line = data.loc[g]
        if gz == 2.0 or gz == 1.0:
            dur_gazeNH = True
            # get the gazed at object
            curr_gazeNH = curr_line.namesNH
        # if you are currently in a gaze:
        if dur_gazeNH:
            # if you are currently having the correct element, add the position
            if curr_line.processedCollider_NH_name == curr_gazeNH:
                hon_posNH = hon_posNH + [
                    [
                        curr_line["processedCollider_NH_hitPointOnObject_x"], 
                        curr_line["processedCollider_NH_hitPointOnObject_y"], 
                        curr_line["processedCollider_NH_hitPointOnObject_z"]
                    ]
                ]
            distNH = distNH + [
                np.array(
                    [
                        curr_line["eyePositionCombinedWorld_x"],
                        curr_line["eyePositionCombinedWorld_y"],
                        curr_line["eyePositionCombinedWorld_z"],
                    ]
                )
            ]

        # once the gaze is over, take the avg_dist
        if gz == -2.0 or gz == -1.0:
            hon_posNH = [np.nanmean(hon_posNH, axis=0)] * len(distNH)
            # calculate to distance at this timpoint
            distNH = [
                np.linalg.norm(hon_posNH[c] - distNH[c]) for c in range(len(distNH))
            ]
            all_distNH = all_distNH + distNH
            # average distance during the gaze event
            avg_distNH = avg_distNH + [np.nanmean(distNH)] * len(distNH)

            # reset everything:
            distNH = []
            hon_posNH = []
            dur_gazeNH = False

        # if there are parts that are neither gaze nor saccade:
        if (
            (not dur_gazeNH)
            and (gz not in [2.0, 1.0])
            and (len(all_distNH) + len(distNH) != g + 1)
        ):
            all_distNH = all_distNH + [np.nan]
            avg_distNH = avg_distNH + [np.nan]

        if len(all_distNH) + len(distNH) != g + 1:
            display(g)

    if dur_gazeNH:
        hon_posNH = [np.nanmean(hon_posNH, axis=0)] * len(distNH)
        # calculate to distance at this timpoint
        distNH = [np.linalg.norm(hon_posNH[c] - distNH[c]) for c in range(len(distNH))]
        all_distNH = all_distNH + distNH
        # average distance during the gaze event
        avg_distNH = avg_distNH + [np.nanmean(distNH)] * len(distNH)

    # add them to for_eye
    data["distance_NH"] = all_distNH
    data["avg_dist_NH"] = avg_distNH

    # save data
    data = pd.DataFrame(data)
    
    
    # save df:
    # 1 == MAD_woBig
    
    if define_intervals == 1:
        # save interval
        saveName = file.replace("data_resampled.csv","data_correTS_mad_wobig.csv")
        data.to_csv(os.path.join(data_savepath_intermediateP, saveName), index = False)
        
    # 2 == 10_sec
    else:
        print("why are you using the 10 sec interval, it is not fully adapted in this script")

    return data

# Loop over all files and apply all above processing functions

In [59]:
print('start')
# useful overviews
noFilePartList = [len(PartList)]
missingFiles = pd.DataFrame(columns=['Participant', 'Session'])

# Initialize counter for filter application failures
filterSG_not_applied_count = 0

# Initialize a list to store the results
overviewFilterSG = []


# loop code over all participants in participant list
parts_pbar = tqdm(
    iterable=PartList,
    total=len(PartList),
    desc="participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

for indexPart, currentPart in enumerate(parts_pbar):
#     currentPart = PartList[indexPart]
    print(f'Participant {indexPart} - participant ID {currentPart}')
    
    sessions = list(range(1,6))
#     sessions = list(range(2,3))
    
    session_pbar = tqdm(
        iterable=sessions,
        total=len(sessions),
        desc="sessions________",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )
    
    
    # loop over recording sessions (should be 5 for each participant)
#     for indexSess in range(1, 6):
#     for indexSess in range(1, 2):
    for indexSess in session_pbar:

        # get all eye tracking sessions and loop over them (amount of ET files can vary)
        fileNames = [f for f in os.listdir(data_savepath_resampled) if f.startswith(f'{currentPart}_Session_{indexSess}') and f.endswith('_data_resampled.csv')]
#         print(dirSess)
#         print(f'Session {indexSess}')
        
        if len(fileNames)< 3:
            hMF = pd.DataFrame({'Participant': [currentPart], 'Session': [indexSess], 'onlyExistingFilesInSession': [fileNames]})
            missingFiles = pd.concat([missingFiles, hMF], ignore_index=True)
            

        # runs if files exist

############ Main part starts here ######################################################################################
        # To use it for later!!!
        # based on remodnav --> has almost the same length as our filter

        # files progress bar
        file_pbar = tqdm(
            iterable=fileNames,
            total=len(fileNames),
            desc="📂 ET files____________",
            dynamic_ncols=True,
            bar_format=B_FORMAT,
        )



            # loop necessary for displaying properly the progressbar with multiprocessing
            # source: https://stackoverflow.com/a/40133278

#             info_list = []

        for file in file_pbar:

            print('process file ', file)

            # read in data
            data = pd.read_csv(os.path.join(data_savepath_resampled, file))

            # do the velocity calculation
            data = velocity_calc(file, data)
            
            # add validData variable for indexing of valid (non nan) eye tracking data
            data = id_valid_data(file, data)

            # cut off max velocity threshold
            data = max_vel_threshold(file, data)

            # apply Savitzky Golay filter
            data = savitzky_golay(file, data)
            
            
            # calculate the segmentation interval
            data, int_data = calc_mad_wobig_interval(file, data)

            # calculate the MAD saccade
            data, int_data = mad_saccade(file, data, int_data)

            # add the threshold to the data df
            data, int_data = add_threshold(file, data, int_data)

            # define fixations
            data, int_data = define_fixations(file,data,int_data)

            
            # identify the index, during which the first event exceeding the min event length starts
            event_index = identify_first_event_index(file, data)

            # correct small events
            data, nr_rm_rows = correct_small_events(file, data, event_index )

            # Correct is an event is too small.
            data = get_events_len_dist(file, data, event_index, nr_rm_rows)


            # adjust the distances
            data = adjust_distance(file, data)

            # adjust the distances for NH variables
            data = adjust_distance_NH(file, data)


            del data             



if len(missingFiles)>0:
    
    print("missing files detected")
    print(missingFiles)
    
else:
    print("All files were found")
    

start


📄 0 of 26 participants processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

Participant 0 - participant ID 1004


📄 0 of 5 sessions________ processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

📄 0 of 3 📂 ET files____________ processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

...

process file  1080_Session_5_ET_3_data_resampled.csv


📄 0 of 54010 ⌚ timestamps from file 1080_Session_5_ET_3_data_resampled.csv processed:            
            …

📄 0 of 54010 ⌚ timestamps from file 1080_Session_5_ET_3_data_resampled.csv processed:            
            …

C:\Users\jawalter\anaconda3\envs\graphs\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: Mean of empty slice
C:\Users\jawalter\anaconda3\envs\graphs\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: Mean of empty slice
C:\Users\jawalter\anaconda3\envs\graphs\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: Mean of empty slice
C:\Users\jawalter\anaconda3\envs\graphs\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: Mean of empty slice


All files were found


# Length: Correct for long Events (outliers)

In [7]:
define_intervals

1

In [8]:
# perform median absolute deviation to find outliers in the event durations:
# https://hausetutorials.netlify.app/posts/2019-10-07-outlier-detection-with-median-absolute-deviation/

# Ashima:
# Finally, we rejected all fixations with a duration greater than 3.5 times the median absolute deviation of the
# population fixation duration...


plotting = False
g_len = []
s_len = []

######################################### start file loop

print('start round 1')
# useful overviews
noFilePartList = [len(PartList)]
missingFiles = pd.DataFrame(columns=['Participant', 'Session'])



# loop code over all participants in participant list
parts_pbar = tqdm(
    iterable=PartList,
    total=len(PartList),
    desc="participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

for indexPart, currentPart in enumerate(parts_pbar):
#     currentPart = PartList[indexPart]
    print(f'Participant {indexPart} - participant ID {currentPart}')
    
    sessions = list(range(1,6))
#     sessions = list(range(2,3))
    
    session_pbar = tqdm(
        iterable=sessions,
        total=len(sessions),
        desc="sessions________",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )
    
    
    # loop over recording sessions (should be 5 for each participant)
#     for indexSess in range(1, 6):
#     for indexSess in range(1, 2):
    for indexSess in session_pbar:

        # get all eye tracking sessions and loop over them (amount of ET files can vary)
        fileNames = [f for f in os.listdir(data_savepath_intermediateP) if f.startswith(f'{currentPart}_Session_{indexSess}') and f.endswith('_data_correTS_mad_wobig.csv')]
#         print(dirSess)
#         print(f'Session {indexSess}')
        
        if len(fileNames)< 3:
            hMF = pd.DataFrame({'Participant': [currentPart], 'Session': [indexSess], 'onlyExistingFilesInSession': [fileNames]})
            missingFiles = pd.concat([missingFiles, hMF], ignore_index=True)
            

        # files progress bar
        file_pbar = tqdm(
            iterable=fileNames,
            total=len(fileNames),
            desc="📂 ET files____________",
            dynamic_ncols=True,
            bar_format=B_FORMAT,
        )

            # loop necessary for displaying properly the progressbar with multiprocessing
            # source: https://stackoverflow.com/a/40133278

#             info_list = []

        for file in file_pbar:

            print('process file ', file)

            # read in data
            data = pd.read_csv(os.path.join(data_savepath_intermediateP, file))

    ######################## start main code here:
            
            # only consider rows that have good eye tracking data

            data = data[data['validData'] == True]

            # separate between gaze and saccade
            gaze = data[data["events"] == 2.0]
            sacc = data[
                data["events"] == 1.0
            ]  # as we do not always have 1.0 in the data, we take the end

            g_len = g_len + gaze["length"].tolist()
            s_len = s_len + sacc["length"].tolist()
            
                    
if len(missingFiles)>0:
    
    print("missing files detected in part 1")
    print(missingFiles)
    
else:
    print("All files were found")
                
#########################

# median of the absolute deviation (* 1.4826 when assuming normal distribution):
gaze_mad = np.nanmedian(abs(g_len - np.nanmedian(g_len))) * 1.4826
sacc_mad = np.nanmedian(abs(s_len - np.nanmedian(s_len))) * 1.4826

# calculate the median to reject data
g_len_med = np.nanmedian(g_len)
s_len_med = np.nanmedian(s_len)


################# start second loop here

print('start round 2')
# useful overviews
noFilePartList = [len(PartList)]
missingFiles = pd.DataFrame(columns=['Participant', 'Session'])



# loop code over all participants in participant list
parts_pbar = tqdm(
    iterable=PartList,
    total=len(PartList),
    desc="participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

for indexPart, currentPart in enumerate(parts_pbar):
#     currentPart = PartList[indexPart]
    print(f'Participant {indexPart} - participant ID {currentPart}')
    
    sessions = list(range(1,6))
#     sessions = list(range(2,3))
    
    session_pbar = tqdm(
        iterable=sessions,
        total=len(sessions),
        desc="sessions________",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )
    
    # loop over recording sessions (should be 5 for each participant)
#     for indexSess in range(1, 6):
#     for indexSess in range(1, 2):
    for indexSess in session_pbar:

        # get all eye tracking sessions and loop over them (amount of ET files can vary)
        fileNames = [f for f in os.listdir(data_savepath_intermediateP) if f.startswith(f'{currentPart}_Session_{indexSess}') and f.endswith('_data_correTS_mad_wobig.csv')]
#         print(dirSess)
#         print(f'Session {indexSess}')
        
        if len(fileNames)< 3:
            hMF = pd.DataFrame({'Participant': [currentPart], 'Session': [indexSess], 'onlyExistingFilesInSession': [fileNames]})
            missingFiles = pd.concat([missingFiles, hMF], ignore_index=True)
            
        
        # runs if files exist

        # files progress bar
        file_pbar = tqdm(
            iterable=fileNames,
            total=len(fileNames),
            desc="📂 ET files____________",
            dynamic_ncols=True,
            bar_format=B_FORMAT,
        )

            # loop necessary for displaying properly the progressbar with multiprocessing
            # source: https://stackoverflow.com/a/40133278

#             info_list = []

        for file in file_pbar:

            print('process file ', file)

            # read in data
            data = pd.read_csv(os.path.join(data_savepath_intermediateP, file))

    ################ main code starts here#######

            # only consider rows that have good eye tracking data
            
            
            invalid = data[data['validData'] == False]
        
            data = data[data['validData'] == True]


            # separate between gaze and saccade
            gaze = data[~data["isFix"].isnull()]
            sacc = data[data["isFix"].isnull()]

            # Deviations:
            # then to calculate the deviations using the entire df for easy lookup later
            gaze_mad_z = abs(gaze["length"].tolist() - g_len_med) / gaze_mad
            sacc_mad_z = abs(sacc["length"].tolist() - s_len_med) / sacc_mad

            # get the outliers:
            gaze_mad_z[gaze_mad_z > 3.5] = np.nan
            sacc_mad_z[sacc_mad_z > 3.5] = np.nan

            # save the data:
            data = pd.concat([gaze, sacc, invalid])
            data["long_events"] = gaze_mad_z.tolist() + sacc_mad_z.tolist() + [np.nan] * len(invalid)
            data = data.sort_index()

            # save df:
            # 1 == MAD_woBig

            if define_intervals == 1:
                # save interval
                saveName = file.replace("data_correTS_mad_wobig.csv", "data_processed_gazes.csv")
                data.to_csv(os.path.join(data_savepath_processed_gazes, saveName), index = False)
            # 2 == 10_sec
            else:
                print("why are you using the 10 sec interval, it is not fully adapted in this script")
                    
if len(missingFiles)>0:
    
    print("missing files detected in part 2")
    print(missingFiles)
    
else:
    print("All files were found")
        

start round 1


📄 0 of 26 participants processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

Participant 0 - participant ID 1004


📄 0 of 5 sessions________ processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

📄 0 of 3 📂 ET files____________ processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

...

process file  1080_Session_3_ET_1_data_correTS_mad_wobig.csv
process file  1080_Session_3_ET_2_data_correTS_mad_wobig.csv
process file  1080_Session_3_ET_3_data_correTS_mad_wobig.csv


📄 0 of 3 📂 ET files____________ processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

process file  1080_Session_4_ET_1_data_correTS_mad_wobig.csv
process file  1080_Session_4_ET_2_data_correTS_mad_wobig.csv
process file  1080_Session_4_ET_3_data_correTS_mad_wobig.csv


📄 0 of 3 📂 ET files____________ processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

process file  1080_Session_5_ET_1_data_correTS_mad_wobig.csv
process file  1080_Session_5_ET_2_data_correTS_mad_wobig.csv
process file  1080_Session_5_ET_3_data_correTS_mad_wobig.csv
All files were found
